**USEFUL LINKS**

DATA DICTIONARY : http://bt.mta.info/wiki/Developers/SIRIMonitoredVehicleJourney

STATIC DATA RESOURCES : http://web.mta.info/developers/developer-data-terms.html#data

DATA FEEDS INFO: https://api.mta.info/#/landing

DEVELOPER RESOURCES: https://new.mta.info/developers

----

In [30]:
import pandas as pd

import os
import requests
import json

In [228]:
stops = pd.read_csv("./secrets/bus/stops.txt")

In [233]:
t_stops = stops[stops["stop_name"].str.contains("W 125 ST/MAL")]

In [234]:
t_stops

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
954,402507,W 125 ST/MALCOLM X BL,NaN,40.808026,-73.945964,NaN,NaN,0,NaN
980,402659,W 125 ST/MALCOLM X BL,NaN,40.807514,-73.945026,NaN,NaN,0,NaN


In [253]:
api_key = "API KEY HERE"
stop_id = "402507"

url = f"http://bustime.mta.info/api/siri/stop-monitoring.json?key={api_key}&OperatorRef=MTA&MonitoringRef={stop_id}"

response = requests.request("GET", url)

# Create Python object from JSON string data
obj = json.loads(response.text)
  
# Pretty Print JSON
json_formatted_str = json.dumps(obj, indent=4)

In [254]:
closest_busses = obj["Siri"]["ServiceDelivery"]["StopMonitoringDelivery"][0]['MonitoredStopVisit'][0:3]

In [255]:
bus = closest_busses[0]

In [256]:
# Destination
bus['MonitoredVehicleJourney']['DestinationName']

'W. HARLEM BWAY - 125 via 3 AV'